# Introduction to the dataset
each record is a movie showed in TV in 1992. In each record we have the following information:
* **NETWORK**: broadcasting network (ABN, BBS, or CBC)
* **MONTH**: 1=January, 2=February, ..., 12=December
* **DAY**: 1=Monday, ..., 7=Sunday
* **RATING**: rate (from 0 to 100) for movie
* **FACT**: 1=based on true events, 0=fictional
* **STARS**: number of actors or actress paid over $300,000
* **PREVIOUS RATING**: rate for program immediately preceding movie on same network
* **COMPETITION**: average rates received by the two competing networks during the movie's broadcast

let's import [numpy](http://numpy.org) and [matplotlib](http://matplotlib.org)

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
random.seed(1)

# Importing the data

The following lines allow us to open a CSV file, parse the lines and load it in a numpy array. Remember, next days you will see more optimal solutions to load data (e.g. **pandas**)!

In [ ]:
f=file('Colonial_Broadcasting_Company_dataset.csv','r')

In [ ]:
tt=[]
for i,el in enumerate(f):
    if i==0:#if we are on the first line, we store the columns' name in an array
        col_name=el.strip().split(',')
    else:#
        tt_temp=[]
        for el1 in el.strip().split(','):
            try:
                tt_temp.append(float(el1))
            except:
                tt_temp.append(el1)        
        tt.append(tt_temp)
# tt=array(tt)

In [ ]:
f.close()

In [ ]:
tt=array(tt,object)

In [ ]:
tt[-10:,:]

# First explorations
## Some exercises
Try to anser the following questions:

What are the mean values of each columns? Try to use the less code that you can and remember the Zen of Python

What are the standard-deviations of the third column?

What are the 87.2th percentile of each columns? [Hint: try to search on google numpy percentile]

Try to answer previous questions but on the sample obtained only for Network 'ABN'

Are you able to say what is the mean rating for each network?

and to plot them?

Are you able to plot the histogram of the movies ratings?

# Something new: [boxplot](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.boxplot)

In [ ]:
boxplot([list(tt[tt[:,1]=='"ABN"',4]),list(tt[tt[:,1]=='"CBC"',4]),list(tt[tt[:,1]=='"BBS"',4])],labels=['ABN','CBC','BBS'])
ylabel('rating')

Are you able to comment the previous plot?

### a more "pythonic" solution

In [ ]:
network=set(tt[:,1])

In [ ]:
boxplot([list(tt[tt[:,1]==netw,4]) for netw in network],labels=network)
ylabel('rating')

### Exercise
Use the boxplot to show ratings for the following two groups: 
* movies with at least a stars 
* movies without stars

Use the boxplot to show ratings for the following two groups: 
* fictional-movies 
* fact-movies

# Correlation
With a plot try explore if there is any association between the variable <i>rating</i> and the variable <i>previous rating</i>

### [Pearson](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html#scipy.stats.pearsonr) Correlation

In [ ]:
from scipy.stats import pearsonr

In [ ]:
pearsonr(tt[:,4],tt[:,7])

Are you able to comment the previous result? Hint: <i>?pearsonr</i> or <i>help(pearsonr)</i>

### [Spearman](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html) Correlation


In [ ]:
from scipy.stats import spearmanr

In [ ]:
spearmanr(tt[:,4],tt[:,7])

### Exercise
Try to answer the following question: What is the correlation between the rating a movie received on a network and the mean of the rating the two competing networks received in the same time. (Use all the tool that you want)

### Exercise

In [ ]:
ya=random.rand(237)*50
yb=exp(2.5-ya+random.randn(237)*10)

we define two arrays of the same length: **ya** and **yb**. Plot one vs. the other. Do you think they are correlated?

Try to quantify the correlation:

### Exercise

In [ ]:
yc1=random.rand(123)*2
yc2=random.rand(17)*2
yd=hstack([11.+yc1+random.randn(123)*1.5,11.+yc2+random.randn(17)*50.])
yc=hstack([yc1,yc2])

we define two arrays of the same length: **yc** and **yd**. Are they correlated?

# Comparing two samples: KS
### Exercise
Use the Kolmogorov-Smirnov test to say if movies with at least a star receive different ratings from moviews without stars.

Let's say that for some reasons we do not believe the result of this test (e.g. data do not meet the assumptions). Therefore, you would like to use a boostrap method to evaluate the KS statistic previously evaluated (i.e. the statistic that you get is it a matter of chance or not?)

In [ ]:
rating=tt[:,4].copy()
star=tt[:,6].copy()

In [ ]:
from scipy.stats import ks_2samp

In [ ]:
res=[]
for el in range(1000):
    shuffle(star)
    res.append(ks_2samp(rating[star<1],rating[star>=1])[0])

In [ ]:
hist(res,20)

In [ ]:
print 'more than %g reshuffled samples (over %g) have a KS statistic larger than %g'%(sum(res>ks_2samp(tt[tt[:,6]<1,4],tt[tt[:,6]>=1,4])[0]),len(res),ks_2samp(tt[tt[:,6]<1,4],tt[tt[:,6]>=1,4])[0])

### Exercise
Are the ratings for movies based on true events different from the fictional ones?

# Linear Regressions
A useful library to perform **linear regressions** and other statistical procedures is [StatsModels](http://statsmodels.sourceforge.net/). To import it:

In [ ]:
import statsmodels.api as sm

## [Ordinary Least Squares](http://statsmodels.sourceforge.net/devel/generated/statsmodels.regression.linear_model.OLS.html)
Let's try to evaluate the relationship between the movies **rating** and the following two explanatory variables:
* **previous rating**, rating for program immediately preceding movie on same network
* **competition**, average of ratings received by the two competing networks during the movie's broadcast

In [ ]:
res=sm.OLS(tt[:,4].astype(float),tt[:,[-2,-1]].astype(float)).fit()

In [ ]:
print res.summary()

#### exercise
are you able to comment the results? Are they good or not?

A useful visualization to evaluate the performance of the model is the following

In [ ]:
plot(res.fittedvalues,tt[:,4],'o')
plot([0,30],[0,30],'r')
ylabel('real values')
xlabel('predict values')
xlim(5,25)
ylim(5,25)

### Adding the intercept
Previous model that we tested was **rating ~ previous rating + competition**. We would like to add an intercept to our model. To do so in statsmodel we have to add a list of ones to the explenatory variables with the command **add_constant**

In [ ]:
from statsmodels.tools import add_constant

In [ ]:
res=sm.OLS(tt[:,4].astype(float),add_constant(tt[:,[-2,-1]].astype(float))).fit()

In [ ]:
print res.summary()

#### Exercise
Is this model better than the previous? How could you say that?

#### Exercise
Are the variables used in this model all useful? What variable will you choose to drop from the model? Fit that model and evaluate it.

#### Exercise
With this model plot the real data and the fitted linear model. (Remember, there are several ways to do. The one that use some of the statsmodel features will give in next line).

In [ ]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [ ]:
prstd, iv_l, iv_u = wls_prediction_std(res)
plot(tt[:,-2], tt[:,4], 'bo', label="real data")
plot(tt[:,-2], res.fittedvalues, 'r--', label="OLS")
plot(tt[:,-2], iv_l, 'r--')
plot(tt[:,-2], iv_u, 'r--')
xlim(0,30)
ylim(8,20)

#### Exercise
Are you able to use all the variables in the dataset as predictors of the movie rating? Try it to use as many variables as you can.

Would you keep all of them?

### Dummy variables
Categorical variables could be not used in linear model. A workaround is to create a **dummy variables**. For instance, consider the second column of our dataset. It is filled by 'ABN', 'BBS', or 'CBC'. We could create a further column which values is one if the newtork is 'ABN' and zero otherwise. In this way we have a numerical variables that could be a useful predictor in our linear model.

In [ ]:
ABN=array([array([el=='"ABN"']) for el in tt[:,1]],float)

In [ ]:
tt_dummy=append(tt,ABN,axis=1)

In [ ]:
res=sm.OLS(tt_dummy[:,4].astype(float),add_constant(tt_dummy[:,[2,3,5,6,7,8,9]].astype(float))).fit()

In [ ]:
print res.summary()

#### Exercise
Create a new dummy variable for network BBS and refit previous model with this new variable

#### Exercise
Create the following new dummy variables and fit a model with all the predictors:
* OCT if month==10
* DEC if mont==12
* APR-MAY if month==4 or month==5
* MON if day==1
* SUN if day==7

#### Exercise
Answer the following questions using the previous model: 
* you are considering to change the network programmation by replacing a show at 8pm with rating 17.5 with another with rating 20.  What would be the expected change in rating for the show at 9pm?
* other scenario: what if ABN and BBS schedule different programs, each of which is expected to rate 2 rating points higher?